# How to Save and Restore the Metagraph
`tf.train.Supervisor.managed_session` makes it easy to save and load the *values* of the trainable variables, but not the model itself.  It's impractical to rebuild the model from scratch when it's time to test or deploy it, however, so that's what we'll explore in this notebook.

Note: This is for a `tf.train.Supervisor.managed_session` only.  If you're using a different kind of session, you may need to call `tf.train.export_meta_graph()`.

## Tidy up First
Delete any old save files before we start.

In [1]:
import tensorflow as tf
NOTEBOOK_DIR = 'graph-save-load'
try:
    tf.gfile.DeleteRecursively(NOTEBOOK_DIR)
except:
    print('Already clean; nothing to do.')

## Create the Computation Graph
Just need a few dummy variables and an operation. 

In [2]:
import tensorflow as tf
g = tf.Graph()

with g.as_default():
    weights = tf.get_variable(name="tf_weights", 
                            shape=[3], 
                            initializer=tf.random_normal_initializer(mean=0, stddev=1, seed=None, dtype=tf.float32),
                            dtype=tf.float32)
    biases = tf.get_variable(name="tf_biases", 
                            shape=[3], 
                            initializer=tf.constant_initializer(3.0),
                            dtype=tf.float32)
    wb = weights * biases

## Create Collections for the Variables We Need to Save
`tf.train.Supervisor.managed_session` actually *does* save the model automatically.  It saves all the trainable variables in a collection called "trainable_variables".  The problem is that a TensorFlow collection is just a list of unnamed tensors, not a dictionary.  In order to make use of it, we need to remember the order in which variables were added to it.  This makes for very error-prone code that is hard to maintain.

So, we'll ignore that and create our own collections--generally, one for each variable.  We can, of course, store a list of variables in a single collection when it makes sense to do so, but we don't need to pile everything together in one place like `managed_session` does.

In [3]:
g.add_to_collection('w', weights)
g.add_to_collection('b', biases)
g.add_to_collection('wb', wb)

## Run a "Training Session" to Create a Save File
We're not training anything in this notebook, but this where we would define `train_op` in a real experiment.  For this notebook, we'll just initialize vars, look at them, and then save.

In [4]:
import os
sv = tf.train.Supervisor(logdir=NOTEBOOK_DIR, graph=g)
save_path = os.path.join(NOTEBOOK_DIR, 'model.ckpt')
with sv.managed_session() as s:
    
    # Peek at the values in the (automatically initialized) tensors.
    # Note that running wb will also run its dependencies (weights and b) automatically
    print(s.run([wb]))
    
    # List all the collections in this graph
    print(g.get_all_collection_keys())
    
    # Save the metagraph and the graph
    sv.saver.save(s, save_path)

[array([ 3.84819174, -4.13745785, -2.05410314], dtype=float32)]
['savers', 'w', 'init_op', 'local_init_op', 'variables', 'trainable_variables', 'wb', 'b', 'ready_op']


## Restart the Notebook Kernel
Clean slate.  TensorFlow state reset.

In [ ]:
os._exit(0)

## Reload the Graph

In [1]:
import tensorflow as tf
import os
NOTEBOOK_DIR = 'graph-save-load'
metagraph_file = os.path.join(NOTEBOOK_DIR, 'model.ckpt.meta')
graph_file = os.path.join(NOTEBOOK_DIR, 'model.ckpt')

with tf.Session() as s:
    new_saver = tf.train.import_meta_graph(metagraph_file)
    new_saver.restore(s, graph_file)
    
    # Let's just restore our weights + biases variable
    wb = tf.get_collection('wb')
    
    x = s.run(wb)
    print(x)

INFO:tensorflow:Restoring parameters from graph-save-load/model.ckpt
[array([ 3.84819174, -4.13745785, -2.05410314], dtype=float32)]
